In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [3]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_nj.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #10912

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long
0,560561,560561,1933087308,4/28/20,299,PPP,"ON TARGET STAFFING, LLC",2050 STATE ROUTE 27,NORTH BRUNSWICK,NJ,...,Temporary Help Services,999 Other,5613 Employment Services,5613.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531
1,560562,560562,2697537303,4/29/20,299,PPP,DIVERSANT LLC,331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR,RED BANK,NJ,...,Employment Placement Agencies,999 Other,5613 Employment Services,5613.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448
2,560563,560563,3055997206,4/16/20,299,PPP,"HEALTHCARE STAFFING & CONSULTING SERVICES, LLC",2535 John F. Kennedy Blvd,Jersey City,NJ,...,Nursing Care Facilities (Skilled Nursing Facil...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761
3,560564,560564,3162367309,4/29/20,299,PPP,BRIAD WENCO LLC,78 OKNER PARKWAY,LIVINGSTON,NJ,...,Limited-Service Restaurants,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090
4,560565,560565,3622487203,4/27/20,299,PPP,J.F. KIELY CONSTRUCTION CO.,1 Radar Way,EATONTOWN,NJ,...,"Highway, Street, and Bridge Construction",999 Other,"2373 Highway, Street, and Bridge Construction",2373.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398


In [4]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [5]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan 

,LoanNumber,full_add,Lat,Long
10926,5141627206,"72 B Pearl Street, North Plainfield, NJ",NaN,NaN
20972,1365307303,"823 East GATE DR, MOUNT LAUREL, NJ",NaN,NaN
30139,2104137710,"775 Rt 1 S, Edison, NJ",NaN,NaN


In [6]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #10904

,LoanNumber,full_add,Lat,Long
0,1933087308,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531
1,2697537303,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448
2,3055997206,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761
3,3162367309,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090
4,3622487203,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/NJ_fips_scraped.csv") 
fips_check_scraped.head()

In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [4]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #40813

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #1239

<h4>FIPS Script

In [7]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [8]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('34')] #NJ
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/98.0.4758.102]


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [9]:
geo_id_list =geo_id 

Length Check

In [10]:
print(len(geo_id_list))

32530


Matching FIPS to DataFrame

In [11]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:1239]

In [12]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

<ipython-input-12-08e9c917fe2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
0,1933087308,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531,[340230062051009]
1,2697537303,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448,[340258013005019]
2,3055997206,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761,[340170041041000]
3,3162367309,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090,[340130204004013]
4,3622487203,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398,[340258126004030]
...,...,...,...,...,...
32528,9754798404,"219 S Main St, Barnegat, NJ",39.745114,-74.231276,[340297340031014]
32529,9776128806,"303 Doremus Ave 303 Doremus Ave, Newark, NJ",40.723252,-74.126015,[340130074001003]
32530,9818008107,"830 GROVE ST, ELIZABETH, NJ",40.655283,-74.218870,[340390307031001]
32531,9876707706,"1256 Indian Head Rd, Toms River, NJ",39.991767,-74.215181,[340297230002002]


Remove brackets

In [13]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

<ipython-input-13-e19831f223ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
0,1933087308,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531,340230062051009
1,2697537303,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448,340258013005019
2,3055997206,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761,340170041041000
3,3162367309,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090,340130204004013
4,3622487203,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398,340258126004030
...,...,...,...,...,...
32528,9754798404,"219 S Main St, Barnegat, NJ",39.745114,-74.231276,340297340031014
32529,9776128806,"303 Doremus Ave 303 Doremus Ave, Newark, NJ",40.723252,-74.126015,340130074001003
32530,9818008107,"830 GROVE ST, ELIZABETH, NJ",40.655283,-74.218870,340390307031001
32531,9876707706,"1256 Indian Head Rd, Toms River, NJ",39.991767,-74.215181,340297230002002


In [14]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check

,LoanNumber,full_add,Lat,Long,FIPS_z
1431,3601787701,"5140 US Hwy 9, howell, NJ",34.251379,-85.453067,NaN
3555,4945367310,"2300 US Highway 1 North, ENGLISHTOWN, NJ",38.930717,-76.973348,NaN
5185,7363288408,"10 Park Pl Bldg 1A Ste 504, Butler, NJ",40.747720,-73.981308,NaN
5762,1603408301,"450 US Highway 1 and 9, Newark, NJ",27.375662,-80.326119,NaN
5772,8351957104,"761 RT 9, BAYVILLE, NJ",43.818420,-73.772538,NaN
8263,3438857305,"1460 US HWY 9 N SUITE 203, WOODBRIDGE, NJ",34.251379,-85.453067,NaN
8763,7836737003,"450 US HIGHWAY 1 AND 9, NEWARK, NJ",27.375662,-80.326119,NaN
12065,2617848702,"749 E. Goepp Street, Bethlehem, NJ",40.626382,-75.361486,NaN
15653,1071227210,"1601 Market Street 2300, Philadelphia, NJ",39.953097,-75.167352,NaN
17215,2984238306,"2720 US Highway 1, Lawrence, NJ",44.499859,-68.146898,NaN


Parse for Merge

In [15]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,1933087308,340230062051009
1,2697537303,340258013005019
2,3055997206,340170041041000
3,3162367309,340130204004013
4,3622487203,340258126004030
...,...,...
32528,9754798404,340297340031014
32529,9776128806,340130074001003
32530,9818008107,340390307031001
32531,9876707706,340297230002002


Read in base data

In [16]:
#if from geocode file
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_nj.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("data/state_data/geo/geo_fips/NJ_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #15796


,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long
0,560561,560561,1933087308,4/28/20,299,PPP,"ON TARGET STAFFING, LLC",2050 STATE ROUTE 27,NORTH BRUNSWICK,NJ,...,Temporary Help Services,999 Other,5613 Employment Services,5613.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531
1,560562,560562,2697537303,4/29/20,299,PPP,DIVERSANT LLC,331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR,RED BANK,NJ,...,Employment Placement Agencies,999 Other,5613 Employment Services,5613.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448
2,560563,560563,3055997206,4/16/20,299,PPP,"HEALTHCARE STAFFING & CONSULTING SERVICES, LLC",2535 John F. Kennedy Blvd,Jersey City,NJ,...,Nursing Care Facilities (Skilled Nursing Facil...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761
3,560564,560564,3162367309,4/29/20,299,PPP,BRIAD WENCO LLC,78 OKNER PARKWAY,LIVINGSTON,NJ,...,Limited-Service Restaurants,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090
4,560565,560565,3622487203,4/27/20,299,PPP,J.F. KIELY CONSTRUCTION CO.,1 Radar Way,EATONTOWN,NJ,...,"Highway, Street, and Bridge Construction",999 Other,"2373 Highway, Street, and Bridge Construction",2373.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32528,593089,593089,9754798404,2/17/21,299,PPS,BAYSIDE DENTISTRY PC,219 S Main St,Barnegat,NJ,...,Offices of Dentists,999 Other,6212 Offices of Dentists,6212.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"219 S Main St, Barnegat, NJ",39.745114,-74.231276
32529,593090,593090,9776128806,4/23/21,299,PPS,LOYALTY TRANSPORT CORP,303 Doremus Ave 303 Doremus Ave,Newark,NJ,...,"General Freight Trucking, Long-Distance, Truck...",999 Other,4841 General Freight Trucking,4841.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"303 Doremus Ave 303 Doremus Ave, Newark, NJ",40.723252,-74.126015
32530,593091,593091,9818008107,7/29/20,299,PPP,EAGLE VELOX TRANS LLC,830 GROVE ST,ELIZABETH,NJ,...,"General Freight Trucking, Local",999 Other,4841 General Freight Trucking,4841.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"830 GROVE ST, ELIZABETH, NJ",40.655283,-74.218870
32531,593092,593092,9876707706,5/1/20,299,PPP,FRAJOVI URBAN AIR LLC,1256 Indian Head Rd,Toms River,NJ,...,NaN,NaN,NaN,NaN,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1256 Indian Head Rd, Toms River, NJ",39.991767,-74.215181


Merge with MainFrame

In [17]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,560561,560561,1933087308,4/28/20,299,PPP,"ON TARGET STAFFING, LLC",2050 STATE ROUTE 27,NORTH BRUNSWICK,NJ,...,999 Other,5613 Employment Services,5613.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531,340230062051009
1,560562,560562,2697537303,4/29/20,299,PPP,DIVERSANT LLC,331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR,RED BANK,NJ,...,999 Other,5613 Employment Services,5613.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448,340258013005019
2,560563,560563,3055997206,4/16/20,299,PPP,"HEALTHCARE STAFFING & CONSULTING SERVICES, LLC",2535 John F. Kennedy Blvd,Jersey City,NJ,...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761,340170041041000
3,560564,560564,3162367309,4/29/20,299,PPP,BRIAD WENCO LLC,78 OKNER PARKWAY,LIVINGSTON,NJ,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090,340130204004013
4,560565,560565,3622487203,4/27/20,299,PPP,J.F. KIELY CONSTRUCTION CO.,1 Radar Way,EATONTOWN,NJ,...,999 Other,"2373 Highway, Street, and Bridge Construction",2373.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398,340258126004030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32528,593089,593089,9754798404,2/17/21,299,PPS,BAYSIDE DENTISTRY PC,219 S Main St,Barnegat,NJ,...,999 Other,6212 Offices of Dentists,6212.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"219 S Main St, Barnegat, NJ",39.745114,-74.231276,340297340031014
32529,593090,593090,9776128806,4/23/21,299,PPS,LOYALTY TRANSPORT CORP,303 Doremus Ave 303 Doremus Ave,Newark,NJ,...,999 Other,4841 General Freight Trucking,4841.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"303 Doremus Ave 303 Doremus Ave, Newark, NJ",40.723252,-74.126015,340130074001003
32530,593091,593091,9818008107,7/29/20,299,PPP,EAGLE VELOX TRANS LLC,830 GROVE ST,ELIZABETH,NJ,...,999 Other,4841 General Freight Trucking,4841.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"830 GROVE ST, ELIZABETH, NJ",40.655283,-74.218870,340390307031001
32531,593092,593092,9876707706,5/1/20,299,PPP,FRAJOVI URBAN AIR LLC,1256 Indian Head Rd,Toms River,NJ,...,NaN,NaN,NaN,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1256 Indian Head Rd, Toms River, NJ",39.991767,-74.215181,340297230002002


If FIPS Encoded

In [ ]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

Drop Extra FIPS Columns

In [ ]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop

In [18]:
fips_merge.loc[fips_merge["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
1431,561992,561992,3601787701,5/1/20,299,PPP,TOWNE STAFFING LLC,5140 US Hwy 9,howell,NJ,...,999 Other,6216 Home Health Care Services,6216.0,NEW JERSEY DISTRICT OFFICE,c $1-2 million,250 to 499,"5140 US Hwy 9, howell, NJ",34.251379,-85.453067,NaN
3555,564116,564116,4945367310,4/30/20,299,PPP,A&G DELIVERY SERVICES LLC,2300 US Highway 1 North,ENGLISHTOWN,NJ,...,999 Other,4232 Furniture and Home Furnishing Merchant Wh...,4232.0,NEW JERSEY DISTRICT OFFICE,c $1-2 million,100 to 249,"2300 US Highway 1 North, ENGLISHTOWN, NJ",38.930717,-76.973348,NaN
5185,565746,565746,7363288408,2/11/21,299,PPS,FIRST ENVIRONMENT INC,10 Park Pl Bldg 1A Ste 504,Butler,NJ,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"10 Park Pl Bldg 1A Ste 504, Butler, NJ",40.747720,-73.981308,NaN
5762,566323,566323,1603408301,1/19/21,299,PPS,SCION HOTELS LLC,450 US Highway 1 and 9,Newark,NJ,...,999 Other,7211 Traveler Accommodation,7211.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"450 US Highway 1 and 9, Newark, NJ",27.375662,-80.326119,NaN
5772,566333,566333,8351957104,4/15/20,299,PPP,QUALITY MEDICAL TRANPORT INC,761 RT 9,BAYVILLE,NJ,...,999 Other,6219 Other Ambulatory Health Care Services,6219.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"761 RT 9, BAYVILLE, NJ",43.818420,-73.772538,NaN
8263,568824,568824,3438857305,4/29/20,299,PPP,NOVISYS LLC,1460 US HWY 9 N SUITE 203,WOODBRIDGE,NJ,...,999 Other,5415 Computer Systems Design and Related Services,5415.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1460 US HWY 9 N SUITE 203, WOODBRIDGE, NJ",34.251379,-85.453067,NaN
8763,569324,569324,7836737003,4/8/20,299,PPP,SCION HOTELS LLC,450 US HIGHWAY 1 AND 9,NEWARK,NJ,...,999 Other,7211 Traveler Accommodation,7211.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"450 US HIGHWAY 1 AND 9, NEWARK, NJ",27.375662,-80.326119,NaN
10926,571487,571487,5141627206,4/27/20,299,PPP,"R & G CONSTRUCTION, CABINET, AND MILLWORK, INC.",72 B Pearl Street,North Plainfield,NJ,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"72 B Pearl Street, North Plainfield, NJ",NaN,NaN,NaN
12065,572626,572626,2617848702,3/30/21,299,PPS,C AND R INDUSTRIAL SERVICES LLC,749 E. Goepp Street,Bethlehem,NJ,...,999 Other,2382 Building Equipment Contractors,2382.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"749 E. Goepp Street, Bethlehem, NJ",40.626382,-75.361486,NaN
15653,576214,576214,1071227210,4/15/20,299,PPP,"TABAS AND ROSEN, P.C.",1601 Market Street 2300,Philadelphia,NJ,...,999 Other,5411 Legal Services,5411.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1601 Market Street 2300, Philadelphia, NJ",39.953097,-75.167352,NaN


In [19]:
geo_NJ_fips = fips_merge

Convert to File

In [20]:
geo_NJ_fips.to_csv("data/state_data/geo/geo_fips/NJ_fips_scraped.csv") 

Review & Compare

In [21]:
geo_NJ_fips = pd.read_csv("data/state_data/geo/geo_fips/NJ_fips_scraped.csv") 
geo_NJ_fips

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,560561,560561,1933087308,4/28/20,299,PPP,"ON TARGET STAFFING, LLC",2050 STATE ROUTE 27,NORTH BRUNSWICK,...,999 Other,5613 Employment Services,5613.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"2050 STATE ROUTE 27, NORTH BRUNSWICK, NJ",40.457578,-74.510531,3.402301e+14
1,1,560562,560562,2697537303,4/29/20,299,PPP,DIVERSANT LLC,331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR,RED BANK,...,999 Other,5613 Employment Services,5613.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"331 NEWMAN SPRINGS RD BLDG 3 STE 350 2ND FLR, ...",40.334716,-74.095448,3.402580e+14
2,2,560563,560563,3055997206,4/16/20,299,PPP,"HEALTHCARE STAFFING & CONSULTING SERVICES, LLC",2535 John F. Kennedy Blvd,Jersey City,...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"2535 John F. Kennedy Blvd, Jersey City, NJ",40.723953,-74.074761,3.401700e+14
3,3,560564,560564,3162367309,4/29/20,299,PPP,BRIAD WENCO LLC,78 OKNER PARKWAY,LIVINGSTON,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,500 or more,"78 OKNER PARKWAY, LIVINGSTON, NJ",40.803235,-74.352090,3.401302e+14
4,4,560565,560565,3622487203,4/27/20,299,PPP,J.F. KIELY CONSTRUCTION CO.,1 Radar Way,EATONTOWN,...,999 Other,"2373 Highway, Street, and Bridge Construction",2373.0,NEW JERSEY DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Radar Way, EATONTOWN, NJ",40.296082,-74.084398,3.402581e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32528,32528,593089,593089,9754798404,2/17/21,299,PPS,BAYSIDE DENTISTRY PC,219 S Main St,Barnegat,...,999 Other,6212 Offices of Dentists,6212.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"219 S Main St, Barnegat, NJ",39.745114,-74.231276,3.402973e+14
32529,32529,593090,593090,9776128806,4/23/21,299,PPS,LOYALTY TRANSPORT CORP,303 Doremus Ave 303 Doremus Ave,Newark,...,999 Other,4841 General Freight Trucking,4841.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"303 Doremus Ave 303 Doremus Ave, Newark, NJ",40.723252,-74.126015,3.401301e+14
32530,32530,593091,593091,9818008107,7/29/20,299,PPP,EAGLE VELOX TRANS LLC,830 GROVE ST,ELIZABETH,...,999 Other,4841 General Freight Trucking,4841.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"830 GROVE ST, ELIZABETH, NJ",40.655283,-74.218870,3.403903e+14
32531,32531,593092,593092,9876707706,5/1/20,299,PPP,FRAJOVI URBAN AIR LLC,1256 Indian Head Rd,Toms River,...,NaN,NaN,NaN,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"1256 Indian Head Rd, Toms River, NJ",39.991767,-74.215181,3.402972e+14


In [22]:
#fill in missing Lat, Long, FIPS
geo_NJ_fips.loc[geo_NJ_fips["FIPS_z"].isnull()] #1241

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
1431,1431,561992,561992,3601787701,5/1/20,299,PPP,TOWNE STAFFING LLC,5140 US Hwy 9,howell,...,999 Other,6216 Home Health Care Services,6216.0,NEW JERSEY DISTRICT OFFICE,c $1-2 million,250 to 499,"5140 US Hwy 9, howell, NJ",34.251379,-85.453067,NaN
3555,3555,564116,564116,4945367310,4/30/20,299,PPP,A&G DELIVERY SERVICES LLC,2300 US Highway 1 North,ENGLISHTOWN,...,999 Other,4232 Furniture and Home Furnishing Merchant Wh...,4232.0,NEW JERSEY DISTRICT OFFICE,c $1-2 million,100 to 249,"2300 US Highway 1 North, ENGLISHTOWN, NJ",38.930717,-76.973348,NaN
5185,5185,565746,565746,7363288408,2/11/21,299,PPS,FIRST ENVIRONMENT INC,10 Park Pl Bldg 1A Ste 504,Butler,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"10 Park Pl Bldg 1A Ste 504, Butler, NJ",40.747720,-73.981308,NaN
5762,5762,566323,566323,1603408301,1/19/21,299,PPS,SCION HOTELS LLC,450 US Highway 1 and 9,Newark,...,999 Other,7211 Traveler Accommodation,7211.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"450 US Highway 1 and 9, Newark, NJ",27.375662,-80.326119,NaN
5772,5772,566333,566333,8351957104,4/15/20,299,PPP,QUALITY MEDICAL TRANPORT INC,761 RT 9,BAYVILLE,...,999 Other,6219 Other Ambulatory Health Care Services,6219.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"761 RT 9, BAYVILLE, NJ",43.818420,-73.772538,NaN
8263,8263,568824,568824,3438857305,4/29/20,299,PPP,NOVISYS LLC,1460 US HWY 9 N SUITE 203,WOODBRIDGE,...,999 Other,5415 Computer Systems Design and Related Services,5415.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1460 US HWY 9 N SUITE 203, WOODBRIDGE, NJ",34.251379,-85.453067,NaN
8763,8763,569324,569324,7836737003,4/8/20,299,PPP,SCION HOTELS LLC,450 US HIGHWAY 1 AND 9,NEWARK,...,999 Other,7211 Traveler Accommodation,7211.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"450 US HIGHWAY 1 AND 9, NEWARK, NJ",27.375662,-80.326119,NaN
10926,10926,571487,571487,5141627206,4/27/20,299,PPP,"R & G CONSTRUCTION, CABINET, AND MILLWORK, INC.",72 B Pearl Street,North Plainfield,...,337 Furniture,3371 Household and Institutional Furniture and...,3371.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"72 B Pearl Street, North Plainfield, NJ",NaN,NaN,NaN
12065,12065,572626,572626,2617848702,3/30/21,299,PPS,C AND R INDUSTRIAL SERVICES LLC,749 E. Goepp Street,Bethlehem,...,999 Other,2382 Building Equipment Contractors,2382.0,NEW JERSEY DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"749 E. Goepp Street, Bethlehem, NJ",40.626382,-75.361486,NaN
15653,15653,576214,576214,1071227210,4/15/20,299,PPP,"TABAS AND ROSEN, P.C.",1601 Market Street 2300,Philadelphia,...,999 Other,5411 Legal Services,5411.0,NEW JERSEY DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1601 Market Street 2300, Philadelphia, NJ",39.953097,-75.167352,NaN
